# Init

In [ ]:
# Init for script use
with open("/home/yu/OneDrive/App/Settings/jupyter + R + Python/python_startup.py", 'r') as _:
    exec(_.read())

import datatable as dt
import glob
import mmcv
import torch
import shutil

from datatable import f
from tqdm.auto import tqdm

dt.init_styles()

WORK_DIR = '/home/yu/OneDrive/Construal'
MODEL_DIR = f'{WORK_DIR}/pretrained_models/mmdetection'
DATA_DIR = f'{WORK_DIR}/data'

os.chdir(WORK_DIR)

# Obj Detect

In [ ]:
os.chdir(MODEL_DIR)
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
os.chdir(WORK_DIR)

## config Model

In [2]:
# Specify the path to model config and checkpoint file
config_file = f'{MODEL_DIR}/configs/lvis/mask_rcnn_x101_64x4d_fpn_sample1e-3_mstrain_1x_lvis_v1.py'
checkpoint_file = f'{MODEL_DIR}/checkpoints/mask_rcnn_x101_64x4d_fpn_sample1e-3_mstrain_1x_lvis_v1-43d9edfe.pth'

# build the model from a config file and a checkpoint file
model_cuda0 = init_detector(config_file, checkpoint_file, device='cuda:0')
model_cuda1 = init_detector(config_file, checkpoint_file, device='cuda:1')

## move jpgs

In [3]:
# get pid of "product design"
ld('pjson', path=DATA_DIR)
pids = pjson[f.category=='Product Design',f.pid].to_list()[0]

"pjson.feather" (167.5 MB) loaded (<1s) (2021-03-09 4:02 PM)


In [4]:
# Copy all jpgs in a project folder to '/Kickstarter Image/project_folder'
def copy_jpg(pid):
    pdir = f'{DATA_DIR}/Kickstarter Data/{pid}'
    for i, file in enumerate(os.listdir(pdir)):
        if file.endswith('.jpg'):
            file_full_path = f'{pdir}/{file}'
            target_dir = f'{DATA_DIR}/Kickstarter Image/{pid}'
            target_path = f'{target_dir}/{file}'

            if not os.path.exists(target_dir):
                os.mkdir(target_dir)

            shutil.copyfile(file_full_path, target_path)

for pid in tqdm(pids):
    copy_jpg(pid)

  0%|          | 0/819 [00:00<?, ?it/s]

In [137]:
# Move depth detection results to a new subfolder
# Only need to do onece
def move_depth_results():
    pids = os.listdir(f'{DATA_DIR}/Kickstarter Image/')

    for pid in tqdm(pids):
        pdir = f'{DATA_DIR}/Kickstarter Image/{pid}'
        for file in os.listdir(pdir):
            if file.endswith('md2.jpg') or file.endswith('md2.npy'):
                file_full_path = f'{pdir}/{file}'
                target_dir = f'{DATA_DIR}/Kickstarter Image/{pid}/depth results'
                target_path = f'{target_dir}/{file}'

                if not os.path.exists(target_dir):
                    os.mkdir(target_dir)

                shutil.move(file_full_path, target_path)
                
# move_depth_results()

  0%|          | 0/43131 [00:00<?, ?it/s]

## Inference

### Unit test

In [ ]:
target = f'{DATA_DIR}/Sharing'

def unit(pid, model):
    pdir = f'{DATA_DIR}/Kickstarter Image/{pid}'
    output = {}
    for i, file in enumerate(os.listdir(pdir)):
        if file.endswith('profile_full.jpg'):
            img = f'{pdir}/{file}'
            res = inference_detector(model, img)
            
            model.show_result(img, res, out_file=f'{target}/example_{i}.jpg')
            
for pid in tqdm(pids[:5]):
    unit(pid, model_cuda1)

### Batch run

In [5]:
# Start Detecting!
def obj_detect(pid, model):
    pdir = f'{DATA_DIR}/Kickstarter Image/{pid}'
    output = {}
    for i, file in enumerate(os.listdir(pdir)):
        if file.endswith('.jpg'):
            img = f'{pdir}/{file}'
            res = inference_detector(model, img)
            
            # get cat_idx and cat_n
            # res[0]: box
            # res[1]: segment
            box_res = {}
            for cat_idx, cat in enumerate(res[0]):
                if len(cat)>0:
                    box_res[cat_idx] = {'n':len(cat),
                                        'prob':cat[:,-1].tolist()}
            
            output[file] = box_res

    return output

x = obj_detect(pids[0], model_cuda0)

/home/yu/OneDrive/Construal/models/mmdetection/mmdet/datasets/utils.py:62: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(


In [229]:
# Start Detecting!
def obj_detect(pid, model):
    pdir = f'{DATA_DIR}/Kickstarter Image/{pid}'
    output = {}
    for i, file in enumerate(os.listdir(pdir)):
        if file.endswith('.jpg'):
            img = f'{pdir}/{file}'
            res = inference_detector(model, img)
            
            # get cat_idx and cat_n
            # res[0]: box
            # res[1]: segment
            box_res = {}
            for cat_idx, cat in enumerate(res[0]):
                if len(cat)>0:
                    box_res[cat_idx] = {'n':len(cat),
                                        'prob':cat[:,-1].tolist()}
            
            output[file] = box_res
            


    # save results
    save_dir = f'{pdir}/object results'
    save_path = f'{save_dir}/mrcnn_lvis.pt'
    if not os.path.exists(save_path):
        os.mkdir(save_dir)
    torch.save(output, save_path)
    
    
device_id = 0
print(f'Using GPU:{device_id}')

half = int(len(pids)/2)
pids = [pids[:half], pids[half:]]

model = init_detector(config_file, checkpoint_file, device=f'cuda:{device_id}')

for pid in tqdm(pids[device_id]):
    obj_detect(pid, model)

## Retreving results for sharing


In [2]:
# get pid of "product design"
ld('pjson', path=DATA_DIR)
pids = pjson[f.category=='Product Design',f.pid].to_list()[0]

"pjson.feather" (167.5 MB) loaded (<1s) (2021-03-21 3:50 PM)


In [3]:
target = f'{DATA_DIR}/Sharing/object detect results'
if not os.path.exists:
    os.mkdir(target)

def save_sharing(pid):
    pdir = f'{DATA_DIR}/Kickstarter Image/{pid}/object results'
    target_dir = f'{target}/{pid}'
    if not os.path.exists(target_dir):
        shutil.copytree(pdir, target_dir)

for pid in tqdm(pids):
    save_sharing(pid)

100%|##########| 819/819 [00:00<00:00, 277878.58it/s]


In [ ]:
import torch

root_dir = '/home/yu/OneDrive/Construal/data/Sharing/object detect results'

detect_res = {}
for i, pid in enumerate(os.listdir(root_dir)):
    res = torch.load(f'{root_dir}/{pid}/mrcnn_lvis.pt')
    detect_res[pid] = res
    

df_objdet = []
for pid, v in detect_res.items():
    for jpg, labels in v.items():
        for label_id, label_counts in labels.items():
            for inst_id, prob in enumerate(label_counts['prob']):
                df_objdet.append((pid, jpg, label_id, inst_id, prob))
df_objdet = dt.Frame(df_objdet, names=['pid', 'jpg', 'label_id', 'inst_id', 'prob'])
sv('df_objdet')

# Image Label distribution

## LVIS distribution (Py)

In [ ]:
LVIS_DATA_DIR = '/home/yu/Data/LVIS'

import json

with open(f'{LVIS_DATA_DIR}/lvis_v1_train.json') as ff:
    lvis_dist = dt.Frame(json.load(ff)['categories'])

lvis_dist.names = {'def': 'definition'}
lvis_dist = lvis_dist[:, 
      [f.id, f.name, f.definition, f.instance_count, f.image_count, f.frequency]]

sv('lvis_dist')

## kick distribution (R)

In [2]:
ld(lvis_dist, force=T) # dist of LVIS
ld(df_objdet) # object detection results

"lvis_dist.feather" (80.4 KB) loaded (0.03 secs) (2021-04-21 1:09 AM)
"df_objdet.feather" (73.1 MB) loaded (0.61 secs) (2021-04-21 1:09 AM)


In [34]:
kick = df_objdet[prob>=0.5, .(kick_freq=.N),
      keyby=.(label_id)
    ][, .(label_id, kick_freq=kick_freq/sum(kick_freq))]

dist = lvis_dist[, .(label_id=id, lvis_freq=instance_count)
    ][kick, on=.(label_id), nomatch=NULL
    ][, ':='(lvis_freq=lvis_freq/sum(lvis_freq))
    ][, ':='(is_kick_more=sign(kick_freq-lvis_freq))]

In [38]:
# plot_ly(dist, x=~label_id, y=~lvis_freq, type='bar', name='LVIS') %>%
#     add_trace(y=~kick_freq, name='Kickstart') %>%
#     plotly::layout(barmode='group')

kick_dist = df_objdet[prob>=0.5, .(inst_count=.N), keyby=.(pid, label_id)
    ][dist, on=.(label_id), nomatch=NULL
    ][, {
      n_labels=uniqueN(label_id)
      n_instances=sum(inst_count)
    
      kick_freq=sum(kick_freq*inst_count)
      kick_freq_norm=kick_freq/n_instances
    
      lvis_freq=sum(lvis_freq*inst_count)
      lvis_freq_norm=lvis_freq/n_instances
    
      abs_freq_diff=abs(sum(kick_freq*inst_count)-sum(lvis_freq*inst_count))
      sign_freq_diff=sign(sum(is_kick_more*inst_count))
      
       
      list(n_labels=n_labels, n_instances=n_instances, kick_freq=kick_freq,
           kick_freq_norm=kick_freq_norm, lvis_freq=lvis_freq, 
           lvis_freq_norm=lvis_freq_norm,
           abs_freq_diff=abs_freq_diff, sign_freq_diff=sign_freq_diff)
      },
      keyby=.(pid)]

In [7]:
ld(lvis_dist)

"lvis_dist.feather" (80.4 KB) loaded (0.02 secs) (2021-05-13 10:28 PM)


In [6]:
fwrite(lvis_dist, 'object_detect_lvis_distribution.csv')

# Test